# MLB Web Scraping Code

In [18]:
import urllib.request
import json
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

### Single Inning Function
NOT USED - See all game below

In [19]:
def pull_inning(year, month, day, game, inning):
    
    #URL
    url="http://www.mlb.com/gdcross/components/game/mlb/year_ID1/month_ID2/day_ID3/ID4/inning/inning_ID5.xml"
    url=url.replace('ID1', year).replace('ID2', month).replace('ID3', day).replace('ID4', game).replace('ID5', inning)
    
    #Dictionary
    columns=['break_y','ax','ay','az', 'year', 'batter']
    all_col={}
    for column in columns:
        all_col[column] = []
      
    #Scrape Data
    gameURL = urllib.request.urlopen(url).read()
    gameSoup = BeautifulSoup(gameURL,"lxml")
    atBat = gameSoup.find_all('atbat')

    for i,batter in enumerate(atBat):
        batter_num = batter['num']
        pitches = atBat[i].find_all('pitch')
        for pitch in pitches:
            try:
                all_col['break_y'].append(pitch['break_y'])
                all_col['ax'].append(pitch['ax'])
                all_col['ay'].append(pitch['ay'])
                all_col['az'].append(pitch['az'])
            except:
                all_col['break_y'].append(0)
                all_col['ax'].append(0)
                all_col['ay'].append(0)
                all_col['az'].append(0)
                
            all_col['year'].append(2016)
            all_col['batter'].append(batter_num)

    one_inning = pd.DataFrame.from_dict(all_col)

    return one_inning



In [20]:
#Test pull_inning function
one_inning = pull_inning('2015','05', '07', 'gid_2015_05_07_lanmlb_milmlb_1', '1')
one_inning.head()

,ax,ay,az,batter,break_y,year
0,-6.108,29.089,-11.977,1,23.8,2016
1,-4.492,30.431,-11.155,1,23.8,2016
2,-5.939,28.772,-11.24,1,23.8,2016
3,-0.597,28.194,-11.091,1,23.8,2016
4,-11.308,24.523,-25.301,1,23.8,2016


### Complete Game
Called in the All games in one day function

In [21]:
def pull_game(year, month, day, game):
    
    #URL
    url="http://www.mlb.com/gdcross/components/game/mlb/year_ID1/month_ID2/day_ID3/ID4/inning/inning_all.xml"
    url=url.replace('ID1', year).replace('ID2', month).replace('ID3', day).replace('ID4', game)
    
    #Dictionary
    columns=['break_y','ax','ay','az', 'year', 'batter','game','inning']
    all_col={}
    for column in columns:
        all_col[column] = []
      
    #Scrape Data
    gameURL = urllib.request.urlopen(url).read()
    gameSoup = BeautifulSoup(gameURL,"lxml")
    atBat = gameSoup.find_all('inning')

    for i,inning in enumerate(atBat):
        inning_num = inning['num']
        batters = atBat[i].find_all('atbat')
        for y, batter in enumerate(batters):
            batter_num=batter['num']
            pitches = batters[y].find_all('pitch')
            for pitch in pitches:
                all_col['year']=year
                all_col['batter'].append(batter_num)
                all_col['game']=game
                all_col['inning'].append(inning_num)
                
                try:
                    all_col['break_y'].append(pitch['break_y'])
                    all_col['ax'].append(pitch['ax'])
                    all_col['ay'].append(pitch['ay'])
                    all_col['az'].append(pitch['az'])
                
                except:
                                                             
                    all_col['break_y'].append(0)
                    all_col['ax'].append(0)
                    all_col['ay'].append(0)
                    all_col['az'].append(0)
                                
    all_game = pd.DataFrame.from_dict(all_col)

    return all_game

In [22]:
#Test pull_game
all_game = pull_game('2015','05', '07', 'gid_2015_05_07_miamlb_sfnmlb_1')
all_game.head(5)

,ax,ay,az,batter,break_y,game,inning,year
0,-13.616,27.956,-20.717,1,23.8,gid_2015_05_07_miamlb_sfnmlb_1,1,2015
1,-15.946,25.664,-28.693,1,23.8,gid_2015_05_07_miamlb_sfnmlb_1,1,2015
2,-7.206,27.878,-16.916,2,23.8,gid_2015_05_07_miamlb_sfnmlb_1,1,2015
3,-10.468,27.344,-22.867,2,23.8,gid_2015_05_07_miamlb_sfnmlb_1,1,2015
4,11.126,21.642,-39.24,2,23.8,gid_2015_05_07_miamlb_sfnmlb_1,1,2015


### All games in one day

In [23]:
year='2015'
month='05'
day='07'

def day_games(year, month, day):
    
    url='http://www.mlb.com/gdcross/components/game/mlb/year_ID1/month_ID2/day_ID3/'
    url = url.replace('ID1', year).replace('ID2', month).replace('ID3', day)
    
    #Dictionary
    columns=['break_y','ax','ay','az', 'year', 'batter','game','inning']
    all_col={}
    for column in columns:
        all_col[column] = []
    
    dayURL = urllib.request.urlopen(url).read()
    daySoup = BeautifulSoup(dayURL,"lxml")
    
    #here we get the game codes for the day's games.
    game_list = []
    for i in daySoup.find_all("a"):
        if i['href'][0:3] == "gid":
            game_list.append(i['href'])
    
    daily_data=pd.DataFrame()
    for game in game_list:
        game_temp = pull_game(year, month, day, game[:-1])
        daily_data=pd.concat([daily_data, game_temp], axis=0)
        
    return daily_data

In [24]:
#test day_games function
day_games1 = day_games('2015','05','07') #20150507
day_games1.head()

,ax,ay,az,batter,break_y,game,inning,year
0,-15.054,33.198,-13.443,1,23.8,gid_2015_05_07_balmlb_nyamlb_1,1,2015
1,-14.45,31.853,-14.716,1,23.8,gid_2015_05_07_balmlb_nyamlb_1,1,2015
2,-13.458,32.781,-13.784,1,23.8,gid_2015_05_07_balmlb_nyamlb_1,1,2015
3,-14.088,30.003,-10.852,1,23.8,gid_2015_05_07_balmlb_nyamlb_1,1,2015
4,-12.363,32.077,-13.891,1,23.8,gid_2015_05_07_balmlb_nyamlb_1,1,2015


### Multiple Days

In [25]:
years = ['2015']#, '2016']
months = ['04','05','06']#,'07','08','09','10']
days = list(range(6,8))


baseball_data=pd.DataFrame()
for year in years:
    for month in months:
        for day in days:
            if day<10:
                day='0'+str(day)
            else:
                day=str(day)
            #print(year,month,day)
            try:
                temp=day_games(year, month, str(day))
                baseball_data=pd.concat([baseball_data, temp], axis=0)
                time.sleep(3)
                
            except:
                pass

    
baseball_data.head()
#baseball_data.to_csv('C:/GTP/Programming/Baseball/Data/baseball2.csv')

,ax,ay,az,batter,break_y,game,inning,year
0,4.618,31.709,-15.654,1,23.7,gid_2015_05_06_arimlb_colmlb_1,1,2015
1,6.857,32.238,-14.609,2,23.8,gid_2015_05_06_arimlb_colmlb_1,1,2015
2,15.369,29.116,-18.365,2,23.8,gid_2015_05_06_arimlb_colmlb_1,1,2015
3,9.218,31.396,-19.457,2,23.8,gid_2015_05_06_arimlb_colmlb_1,1,2015
4,10.144,32.34,-16.929,2,23.7,gid_2015_05_06_arimlb_colmlb_1,1,2015


In [17]:
len(baseball_data)

48510